#### **Creating more custom plots, figures, and graphs.**

The current notebook will generate some figures found in Gagnon at al. 2024. Most of them have been created to highlight some specific results that were not already generated during the actual analysis. *Please note, this notebook doesn't perform any analysis, it simply reuse previously computed results and visualize them.*

**Here is a list of the custom figures that will created with the following cells:**

1. Radar plot showing mean cognitive and behavioral values and stds for all studies combined. Also generate a radar plot for each individual study.
1. Graph network with data coming from the BANDA and GESTE studies labelled. 

In [9]:
# Imports
import os
import math

import matplotlib.pyplot as plt
from matplotlib.pyplot import get_cmap
from matplotlib.colors import rgb2hex
from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections import register_projection
from matplotlib.projections.polar import PolarAxes
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D
import networkx as nx
import numpy as np
import pandas as pd
from scipy.stats import f_oneway
from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.multicomp import pairwise_tukeyhsd

from neurostatx.network.utils import fetch_attributes_df, fetch_edge_data
from neurostatx.network.viz import visualize_network

In [10]:
# Setting up relevant paths.
repository_path = "/Users/anthonygagnon/code/Article-s-Code/" # CHANGE THIS
abcd_base_path = "/Volumes/T7/CCPM/ABCD/Release_5.1/abcd-data-release-5.1/" # CHANGE THIS
geste_base_dir = "/Volumes/T7/CCPM/GESTE/" # CHANGE THIS
banda_dir = '/Volumes/T7/CCPM/BANDA/BANDARelease1.1/' # CHANGE THIS
output_folder = "/Volumes/T7/CCPM/RESULTS_JUNE_24/" # CHANGE THIS
data_dir = f"{output_folder}/fuzzyclustering/"
output_dir = f"{output_folder}/viz/"

# Create output directory if it does not exist.
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [12]:
# Load up the graph network file.
G = nx.read_gml(f"{data_dir}/GraphNetwork.gml")

In [13]:
# Fetch the attributes from the graph network.
attributes_df = fetch_attributes_df(G, attributes='')

# Fetch the edge data. 
edge_df = fetch_edge_data(G)

#### **Generate a radar plot for the global population (all 3 studies).**

In [39]:
def radar_plot(X, labels, output, frame='circle', title="Radar plot",
               cmap='magma'):
    """
    Function to plot a radar plot for all features in the original dataset
    stratified by clusters. T-test between clusters' mean within a feature is
    also computed and annotated directly on the plot. When plotting a high
    number of clusters, plotting of significant annotation is polluting the
    plot, will be fixed in the future.

    Parameters
    ----------
    X : DataFrame
        Input dataset of shape (S, F).
    labels : np.array
        Array of hard membership value (S, ).
    output : str
        Filename of the png file.
    frame : str, optional
        Shape of the radar plot. Defaults to 'circle'. Choices are 'circle'
        or 'polygon'.
    title : str, optional
        Title of the plot. Defaults to 'Radar plot'.
    cmap : str, optional
        Colormap to use for the plot. Defaults to 'magma'. See
        https://matplotlib.org/stable/tutorials/colors/colormaps.html
    """

    # Setting color palette.
    cmap = get_cmap(cmap, len(np.unique(labels)))
    colors = [rgb2hex(cmap(i)) for i in range(cmap.N)]

    # Make labels start at 1 rather than 0, better for viz.
    labels = labels + 1

    # Axis labels.
    var_labels = X.columns.tolist()
    var_labels.append(var_labels[0])

    # Computing ANOVA for each features.
    anova = []
    i = 0
    for col in X.columns:
        f, p = f_oneway(*[X.loc[labels == k, col] for k in np.unique(labels)])
        anova.append(p)
        i += 1

    # Computing mean values for each features for each clusters.
    mean_df = pd.DataFrame()
    i = 0
    for col in X.columns:
        mean = list()
        for k in np.unique(labels):
            mean.append(X.loc[labels == k, col].mean())
        mean_df.insert(i, col, mean)
        i += 1

    # Computing stds for each features for each clusters.
    std_df = pd.DataFrame()
    i = 0
    for col in X.columns:
        std = list()
        for k in np.unique(labels):
            std.append(X.loc[labels == k, col].std())
        std_df.insert(i, col, std)
        i += 1
    max_val = math.ceil((mean_df + std_df).max().max())
    min_val = math.floor((mean_df - std_df).min().min())

    mean_df.insert(i, "Clusters", np.unique(labels))
    std_df.insert(i, "Clusters", np.unique(labels))

    with plt.rc_context(
        {"font.size": 12, "font.weight": "bold", "axes.titleweight": "bold",
         "font.family": "Sans Serif"}
    ):
        fig = plt.figure(figsize=(12, 12))
        ax = fig.add_subplot(111, polar=True)

        # Set radar plot parameters.
        theta = create_radar_plot(len(X.columns), frame=frame)

        for idx, cluster in enumerate(np.unique(labels)):
            values = mean_df.iloc[idx].drop('Clusters').values.tolist()
            values.append(values[0])
            stds = std_df.iloc[idx].drop('Clusters').values.tolist()
            stds.append(stds[0])
            stds_pos = [np.sum(x) for x in zip(values, stds)]
            stds_neg = [s - d for s, d in zip(values, stds)]
            ax.plot(theta, values, c=colors[idx], linewidth=2,
                    label=f'Cluster {cluster}', markersize=4, zorder=3)
            plot = ax.errorbar(theta, values, yerr=stds, fmt='o-',
                               color=colors[idx], linewidth=0,
                               label=f'Cluster {cluster}')
            ax.fill_between(theta, values, stds_pos, alpha=0.2,
                            color=colors[idx], edgecolor='none',
                            label='_nolegend_')
            ax.fill_between(theta, values, stds_neg, alpha=0.2,
                            color=colors[idx], edgecolor='none',
                            label='_nolegend_')

            plot[-1][0].set_color(colors[idx])

    # Add p-values to the plot.
    for i, p in enumerate(anova):
        if 0.01 < p < 0.05:
            ax.text(theta[i], max_val * 0.95, '*', fontsize=24, color='black',
                    weight='bold', rotation=math.degrees(theta[i]) + 90)
        elif 0.001 < p < 0.01:
            ax.text(theta[i], max_val * 0.95, '**', fontsize=24, color='black',
                    weight='bold', rotation=math.degrees(theta[i]) + 90)
        elif p < 0.001:
            ax.text(theta[i], max_val * 0.95, '***', fontsize=24,
                    color='black', weight='bold',
                    rotation=math.degrees(theta[i]) + 90,
                    ha='center', va='center')

    # Set legend and variables parameters.
    legend = ax.legend(np.unique(labels), loc=(0.95, 0.05), title='Profile #',
                       fontsize=14)
    frame = legend.get_frame()
    frame.set_facecolor('white')
    frame.set_edgecolor('black')
    ax.set_thetagrids(theta * 180 / np.pi, var_labels, zorder=1)
    ax.set_rlabel_position(-36)
    ax.set_ylim(min_val, max_val)
    yticks = np.arange(min_val, max_val, 0.5)
    ax.set_yticks(yticks)

    # Set spines and title parameters.
    for spine in ax.spines.values():
        spine.set_color('black')
        spine.set_linewidth(2)
    ax.grid(axis='y', color='lightgray', linewidth=1, zorder=3)
    ax.grid(axis='x', color='lightgray', linewidth=.5, zorder=2)
    ax.set_facecolor('white')
    ax.set_xticklabels(var_labels, fontsize=24, weight='bold')

    ax.set_title(f"{title}", weight='bold', size=24,
                 horizontalalignment='center')

    # Set the position for the labels.
    for label, angle in zip(ax.get_xticklabels(), theta):
        if angle == 0:
            label.set_horizontalalignment('left')
        elif angle == np.pi:
            label.set_horizontalalignment('right')
        elif 0 < angle < np.pi / 2 or angle > 3 * np.pi / 2:
            label.set_horizontalalignment('left')
        else:
            label.set_horizontalalignment('right')

    plt.tight_layout()
    plt.savefig(f"{output}", dpi=300)
    plt.close()


def create_radar_plot(nb_vars, frame='circle'):
    """
    Create a radar chart with `nb_vars` axes.

    Args:
        nb_vars (int):          Number of variables to plot.
        frame (str, optional):  Shape of the radar plot. Defaults to 'circle'.
                                Choices are 'circle' or 'polygon'.

    Returns:
        np.array:               Array of evenly spaced axis angles.
    """

    # Compute evenly spaced axis angles.
    theta = np.linspace(0, 2 * np.pi, nb_vars, endpoint=False)
    theta = np.concatenate((theta, [theta[0]]))

    class RadarTransform(PolarAxes.PolarTransform):

        def transform_path_non_affine(self, path):
            if path._interpolation_steps > 1:
                path = path.interpolated(nb_vars)
            return Path(self.transform(path.vertices), path.codes)

    class RadarAxes(PolarAxes):

        name = 'radar'
        PolarTransform = RadarTransform

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            # Rotate plot to place the first axis at the top.
            self.set_theta_zero_location('N')

        def fill(self, *args, closed=True, **kwargs):
            """Override fill so that line is closed by default.

            Args:
                closed (bool, optional): _description_. Defaults to True.
            """
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            """
            Override plot so that line is closed by default.
            """
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            if x[0] != x[-1]:
                x = np.append(x, x[0])
                y = np.append(y, y[0])
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            if frame == 'circle':
                return Circle((0.5, 0.5), 0.5)
            elif frame == 'polygon':
                return RegularPolygon((0.5, 0.5), nb_vars,
                                      radius=0.5, edgecolor='k')
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

        def _gen_axes_spines(self):
            if frame == 'circle':
                return super()._gen_axes_spines()
            elif frame == 'polygon':

                spine = Spine(axes=self,
                              spine_type='circle',
                              path=Path.unit_regular_polygon(nb_vars))
                spine.set_transform(
                    Affine2D().scale(0.5).translate(0.5, 0.5) + self.transAxes)
                return {'polar': spine}
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)

    return theta

In [14]:
# Assertion that index from the attributes df is the same as the edge df.
assert np.all(attributes_df.index == edge_df.index), "Mismatch in index between attributes and edge data."

In [22]:
# Scale the values by dividing them by the maximum value. Using a loop to avoid hardcoding.
vars = ['Internalization', 'Externalization', 'Stress', 'VA', 'EFPS', 'MEM']

for var in vars:
    attributes_df.loc[:, var] = MinMaxScaler((0, 5)).fit_transform(attributes_df[[var]])

membership = np.argmax(edge_df.values, axis=1)
attributes_df.loc[:, 'profiles'] = membership

In [40]:
radar_plot(attributes_df.loc[:, vars], attributes_df.profiles, title='Combined',
           output=f"{output_dir}/RadarPlotCombined.png", cmap="bone")

In [42]:
# Generating radar plot for each dataset.

abcd_df = attributes_df[attributes_df.cohort == 1]
banda_df = attributes_df[attributes_df.cohort == 2]
geste_df = attributes_df[attributes_df.cohort == 3]

radar_plot(abcd_df.loc[:, vars], abcd_df.profiles, title='ABCD Baseline',
           output=f"{output_dir}/RadarPlotABCD.png", cmap="bone")
radar_plot(banda_df.loc[:, vars], banda_df.profiles, title='BANDA',
           output=f"{output_dir}/RadarPlotBANDA.png", cmap="bone")
radar_plot(geste_df.loc[:, vars], geste_df.profiles, title='GESTE',
           output=f"{output_dir}/RadarPlotGESTE.png", cmap="bone")

#### **Exporting results from the one-way ANOVA between profiles on the raw cognitive and behavioral variables.**

When generating the radar plot, a one-way ANOVA is computed to determine the statistical difference between profiles for each raw variable. However, results are not exported in tabular format but appended to the radar plot. The next cells will compute the ANOVA, and export the results in a table. The exported table will include results from the combined and individual studies. Additionally, Tukey HSD posthoc test will be conducted to further detail the difference in means between each profile.

In [59]:
# Computing the ANOVA for the combined dataset.
anova_combined = []
for var in vars:
    f, p = f_oneway(*[attributes_df.loc[attributes_df.profiles == i, var] for i in np.unique(attributes_df.profiles)])
    anova_combined.append([var, f, p])


In [60]:
# Computing the ANOVA for each dataset.
anova_abcd = []
anova_banda = []
anova_geste = []

for var in vars:
    f, p = f_oneway(*[abcd_df.loc[abcd_df.profiles == i, var] for i in np.unique(abcd_df.profiles)])
    anova_abcd.append([var, f, p])

    f, p = f_oneway(*[banda_df.loc[banda_df.profiles == i, var] for i in np.unique(banda_df.profiles)])
    anova_banda.append([var, f, p])

    f, p = f_oneway(*[geste_df.loc[geste_df.profiles == i, var] for i in np.unique(geste_df.profiles)])
    anova_geste.append([var, f, p])

In [61]:
# Merging into a single Pandas DataFrame.
anova_combined_df = pd.DataFrame(anova_combined, columns=['Variable', 'F_comb', 'p_comb'])
anova_combined_df.set_index('Variable', inplace=True)
anova_abcd_df = pd.DataFrame(anova_abcd, columns=['Variable', 'F_abcd', 'p_abcd'])
anova_abcd_df.set_index('Variable', inplace=True)
anova_banda_df = pd.DataFrame(anova_banda, columns=['Variable', 'F_banda', 'p_banda'])
anova_banda_df.set_index('Variable', inplace=True)
anova_geste_df = pd.DataFrame(anova_geste, columns=['Variable', 'F_geste', 'p_geste'])
anova_geste_df.set_index('Variable', inplace=True)

# Merging the DataFrames.
anova_df = pd.concat([anova_combined_df, anova_abcd_df, anova_banda_df, anova_geste_df], axis=1)
anova_df.to_excel(f"{output_dir}/ANOVA_results.xlsx")


#### **Perform post-hoc Tukey HSD test for pairwise comparison.**

The following cells will perform the Tukey HSD post-hoc test to identify pairwise difference between profiles.

In [62]:
# Add string to profiles label, easy to interpret tables afterwards.
attributes_df.loc[:, 'profiles'] = attributes_df.profiles.apply(lambda x: f"Profile {x+1}")
abcd_df.loc[:, 'profiles'] = abcd_df.profiles.apply(lambda x: f"Profile {x+1}")
banda_df.loc[:, 'profiles'] = banda_df.profiles.apply(lambda x: f"Profile {x+1}")
geste_df.loc[:, 'profiles'] = geste_df.profiles.apply(lambda x: f"Profile {x+1}")

# Tukey's HSD post-hoc test.
tukey_combined = []

for var in vars:
    tukey = pairwise_tukeyhsd(attributes_df[var], attributes_df.profiles, alpha=0.05)
    tukey_combined.append(tukey.summary())

tukey_abcd = []
tukey_banda = []
tukey_geste = []

for var in vars:
    tukey = pairwise_tukeyhsd(abcd_df[var], abcd_df.profiles, alpha=0.05)
    tukey_abcd.append(tukey.summary())

    tukey = pairwise_tukeyhsd(banda_df[var], banda_df.profiles, alpha=0.05)
    tukey_banda.append(tukey.summary())

    tukey = pairwise_tukeyhsd(geste_df[var], geste_df.profiles, alpha=0.05)
    tukey_geste.append(tukey.summary())

In [63]:
# Export the results to Excel.
with pd.ExcelWriter(f"{output_dir}/Tukey_results.xlsx") as writer:
    for i, var in enumerate(vars):
        pd.DataFrame(tukey_combined[i]).to_excel(writer, sheet_name=f"{var}_combined", header=False, index=False)
        pd.DataFrame(tukey_abcd[i]).to_excel(writer, sheet_name=f"{var}_abcd", header=False, index=False)
        pd.DataFrame(tukey_banda[i]).to_excel(writer, sheet_name=f"{var}_banda", header=False, index=False)
        pd.DataFrame(tukey_geste[i]).to_excel(writer, sheet_name=f"{var}_geste", header=False, index=False)

#### **Perform an one-way ANOVA to evaluate differences in mean for each profile across studies**

The following cells will perform a one-way ANOVA between each cohort for each profile for each variable. Additional post-hoc Tukey HSD will also be performed to pinpoint the differences between cohorts.

In [17]:
attributes_df

,pos,Site,Sex,AgeMonths,Ethnicity,Parent_ed1,Parent_ed2,high_edu,edu_groups,Income,...,PTSD,PSYPATHO,cohort,Internalization,Externalization,Stress,VA,EFPS,MEM,profiles
NDAR_INV003RTV85,"[-0.03986787796020508, 0.6829884052276611]",site06,2,131,1,13.0,13.0,13.0,2.0,8.0,...,0,0,1,-0.808537,-0.508119,-0.835551,-0.028635,0.075388,0.075918,2
NDAR_INV005V6D2C,"[0.6076282262802124, 0.6492170095443726]",site10,2,121,3,6.0,0.0,6.0,1.0,999.0,...,0,0,1,-0.712289,-0.621695,-0.780115,-0.358549,-0.036977,-0.262415,3
NDAR_INV007W6H7B,"[-0.33013346791267395, -0.009929507039487362]",site22,1,126,1,19.0,18.0,19.0,5.0,10.0,...,0,0,1,0.355900,-0.586930,0.172099,0.329778,0.138661,0.444795,0
NDAR_INV00BD7VDC,"[-0.48414772748947144, -0.2177594155073166]",site07,1,112,1,20.0,20.0,20.0,5.0,10.0,...,0,1,1,0.034111,-0.323249,0.252535,-0.125655,0.311077,0.495244,2
NDAR_INV00CY2MDM,"[0.2721783518791199, -0.7810755968093872]",site20,1,130,1,15.0,0.0,15.0,3.0,6.0,...,0,1,1,0.117661,2.090632,1.315690,-0.287412,0.037662,-0.708332,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-2400,"[0.49368661642074585, -0.5315656065940857]",0,1,131,1,3.0,5.0,0.0,5.0,39420.0,...,0,1,3,0.647188,0.860618,0.701488,-0.555935,-0.350581,-0.362099,1
sub-11441,"[-0.8550391793251038, 0.05417872965335846]",0,2,130,1,4.0,3.0,0.0,4.0,94900.0,...,0,0,3,-0.825163,-0.490999,-0.649811,0.691489,0.171001,0.351421,2
sub-11442,"[-0.26875224709510803, -0.48823127150535583]",0,2,130,1,4.0,3.0,0.0,4.0,102200.0,...,0,0,3,0.536327,-0.490999,0.025487,0.437418,-0.075040,-0.150013,0
sub-20161,"[0.35089823603630066, -0.5546611547470093]",0,1,151,1,5.0,1.0,0.0,5.0,18250.0,...,0,0,3,2.072545,0.624448,1.495081,0.230475,0.081190,-0.082217,0


In [23]:
# Fetch membership values and attribute them to their main profile.
membership = np.argmax(edge_df.values, axis=1)
attributes_df.loc[:, 'profiles'] = membership

# Compute the differences between the profiles between cohorts.
c1_df = attributes_df[attributes_df.profiles == 0]
c2_df = attributes_df[attributes_df.profiles == 1]
c3_df = attributes_df[attributes_df.profiles == 2]
c4_df = attributes_df[attributes_df.profiles == 3]

# Compute the differences between the profiles between cohorts.
anova_c1 = []
anova_c2 = []
anova_c3 = []
anova_c4 = []

vars = ['Internalization', 'Externalization', 'Stress', 'VA', 'EFPS', 'MEM']

for var in vars:
    f, p = f_oneway(*[c1_df.loc[c1_df.cohort == i, var] for i in np.unique(c1_df.cohort)])
    anova_c1.append([var, f, p])
    f, p = f_oneway(*[c2_df.loc[c2_df.cohort == i, var] for i in np.unique(c2_df.cohort)])
    anova_c2.append([var, f, p])
    f, p = f_oneway(*[c3_df.loc[c3_df.cohort == i, var] for i in np.unique(c3_df.cohort)])
    anova_c3.append([var, f, p])
    f, p = f_oneway(*[c4_df.loc[c4_df.cohort == i, var] for i in np.unique(c4_df.cohort)])
    anova_c4.append([var, f, p])

anova_c1 = pd.DataFrame(anova_c1, columns=['Variable', 'F_c1', 'p_c1'])
anova_c1.set_index('Variable', inplace=True)
anova_c2 = pd.DataFrame(anova_c2, columns=['Variable', 'F_c2', 'p_c2'])
anova_c2.set_index('Variable', inplace=True)
anova_c3 = pd.DataFrame(anova_c3, columns=['Variable', 'F_c3', 'p_c3'])
anova_c3.set_index('Variable', inplace=True)
anova_c4 = pd.DataFrame(anova_c4, columns=['Variable', 'F_c4', 'p_c4'])
anova_c4.set_index('Variable', inplace=True)

# Concatenate the DataFrames.
anova_c = pd.concat([anova_c1, anova_c2, anova_c3, anova_c4], axis=1)
anova_c.to_excel(f"{output_dir}/ANOVA_results_cohorts.xlsx")

In [24]:
# Tukey's HSD post-hoc test.
tukey_c1 = []
tukey_c2 = []
tukey_c3 = []
tukey_c4 = []

for var in vars:
    tukey = pairwise_tukeyhsd(c1_df[var], c1_df.cohort, alpha=0.05)
    tukey_c1.append(tukey.summary())

    tukey = pairwise_tukeyhsd(c2_df[var], c2_df.cohort, alpha=0.05)
    tukey_c2.append(tukey.summary())

    tukey = pairwise_tukeyhsd(c3_df[var], c3_df.cohort, alpha=0.05)
    tukey_c3.append(tukey.summary())

    tukey = pairwise_tukeyhsd(c4_df[var], c4_df.cohort, alpha=0.05)
    tukey_c4.append(tukey.summary())

# Export the results to Excel.
with pd.ExcelWriter(f"{output_dir}/Tukey_results_cohorts.xlsx") as writer:
    for i, var in enumerate(vars):
        pd.DataFrame(tukey_c1[i]).to_excel(writer, sheet_name=f"{var}_c1", header=False, index=False)
        pd.DataFrame(tukey_c2[i]).to_excel(writer, sheet_name=f"{var}_c2", header=False, index=False)
        pd.DataFrame(tukey_c3[i]).to_excel(writer, sheet_name=f"{var}_c3", header=False, index=False)
        pd.DataFrame(tukey_c4[i]).to_excel(writer, sheet_name=f"{var}_c4", header=False, index=False)

#### **Generate a Graph Network file with GESTE and BANDA data labelled.**

The next cells will generate a graph network figure highlight subjects coming from the BANDA and GESTE study. The main purpose of this figure is to evaluated the distribution of both studies across the graph network. Since it is projected onto the ABCD clustering results, we want to ensure that it covers the majority of the graph, and not only specific regions.

In [17]:
# Generate colormap for the different cohorts.
label = attributes_df['cohort'].values

nodes_cmap = []
for i in label:
    if i == 1:
        nodes_cmap.append("darkgrey")
    elif i == 2:
        nodes_cmap.append("red")
    else:
        nodes_cmap.append("orange")

# Create node alpha list.
nodes_alpha = []
for i in nodes_cmap:
    if i == "darkgrey":
        nodes_alpha.append(0.3)
    else:
        nodes_alpha.append(1)

In [18]:
# Visualize the network.
visualize_network(G, output=f'{output_dir}/NetworkCohort.png',
                  weight='membership',
                  subject_node_color=nodes_cmap,
                  subject_alpha=nodes_alpha,
                  colormap='bone_r',
                  title='Cohort labelling.',
                  legend_title='Membership Values')   

#### **Visualize the diagnosis labels from the youth KSADS on the Graph Network.**

The next cells will generate a visualization of the graph network while highlighting participants with a diagnosis reported using the KSADS youth instrument. This is particularly interesting to compare with the distribution of diagnoses reported using the KSADS parent instruments, as it has been suggested that results may vary based on the responder (child or parent).

In [3]:
# Import the GraphNetwork file from the awp folder.
G = nx.read_gml(f"{output_folder}/awp/GraphNetwork_youth.gml")

In [4]:
# Fetch attributes. 
attr = fetch_attributes_df(G, attributes='')

attr['AD_youth'] = attr['AD_youth'].fillna(0)
attr['DD_youth'] = attr['DD_youth'].fillna(0)

In [5]:
nodes_cmap = []
for i in attr['AD_youth'].values:
    if i == 1:
        nodes_cmap.append("orange")
    else:
        nodes_cmap.append("darkgrey")

# Create node alpha list.
nodes_alpha = []
for i in nodes_cmap:
    if i == "darkgrey":
        nodes_alpha.append(0.3)
    else:
        nodes_alpha.append(1)

In [6]:
# Visualize the network.
visualize_network(G, output=f'{output_dir}/NetworkABCD_AD_youth.png',
                  weight='membership',
                  subject_node_color=nodes_cmap,
                  subject_alpha=nodes_alpha,
                  colormap='bone_r',
                  title='ABCD AD Youth',
                  legend_title='AD Youth')

In [7]:
nodes_cmap = []
for i in attr['DD_youth'].values:
    if i == 1:
        nodes_cmap.append("orange")
    else:
        nodes_cmap.append("darkgrey")

# Create node alpha list.
nodes_alpha = []
for i in nodes_cmap:
    if i == "darkgrey":
        nodes_alpha.append(0.3)
    else:
        nodes_alpha.append(1)

In [8]:
# Visualize the network.
visualize_network(G, output=f'{output_dir}/NetworkABCD_DD_youth.png',
                  weight='membership',
                  subject_node_color=nodes_cmap,
                  subject_alpha=nodes_alpha,
                  colormap='bone_r',
                  title='ABCD DD Youth',
                  legend_title='DD Youth')

In [ ]:
# PSYPATHO index.
attr['PSYPATHO_youth'] = attr['PSYPATHO_youth'].fillna(0)

nodes_cmap = []
for i in attr['PSYPATHO_youth'].values:
    if i == 1:
        nodes_cmap.append("orange")
    else:
        nodes_cmap.append("darkgrey")

# Create node alpha list.
nodes_alpha = []
for i in nodes_cmap:
    if i == "darkgrey":
        nodes_alpha.append(0.3)
    else:
        nodes_alpha.append(1)


In [8]:
# Visualize the network.
visualize_network(G, output=f'{output_dir}/NetworkABCD_PSYPATHO_youth.png',
                  weight='membership',
                  subject_node_color=nodes_cmap,
                  subject_alpha=nodes_alpha,
                  colormap='bone_r',
                  title='ABCD PSYPATHO Youth',
                  legend_title='Membership Values')